# 1 Introducción
El siguiente cuaderno realiza la multiplicación de dos vectores, utilizando GPGPU. El resultado se almacena en un tercer vector. El kernel ejecuta en hilos sobre una dimensión (x).

M=αA+B

Para la resolución se aplicaron conceptos de Lenguaje Python, CUDA y el manejo de operaciones aritméticas sobre vectores.

---
# 2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

---
# 3 Desarrollo


In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   5000#@param {type: "number"}
alfa =   1#@param {type: "number"}
# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# CPU - Defino la memoria de los vectores en cpu.
A_cpu = numpy.random.randn( cantidad_N )
A_cpu = A_cpu.astype( numpy.float32() )

B_cpu = numpy.random.randn( cantidad_N )
B_cpu = B_cpu.astype( numpy.float32() )

M_cpu = numpy.random.randn( cantidad_N )
M_cpu = M_cpu.astype( numpy.float32() )

tiempo_ini_cpu = datetime.now()

# CPU - reservo la memoria GPU.
A_gpu = cuda.mem_alloc( A_cpu.nbytes )
B_gpu = cuda.mem_alloc( B_cpu.nbytes )
M_gpu = cuda.mem_alloc( M_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( A_gpu, A_cpu )
cuda.memcpy_htod( B_gpu, B_cpu )
cuda.memcpy_htod( M_gpu, M_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void Multplicacion_Vectores( int n, float alfa, float *X, float *Y, float *M)
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
    M[idx]  = alfa*X[idx] * Y[idx];
  }
}
""") 

# CPU - Genero la función kernel.
kernel = module.get_function("Multplicacion_Vectores")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
dim_hilo = 256
dim_bloque = numpy.int( (cantidad_N+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#Kernel
kernel( numpy.int32(cantidad_N),numpy.float32(alfa), A_gpu, B_gpu, M_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

## GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( M_cpu, M_gpu)

#"""
# CPU - Informo el resutlado.
print( "------------------------------------")
print( "Vector A: " )
print( A_cpu )
print( "------------------------------------")
print( "Vector B: " )
print( B_cpu )
print( "------------------------------------")
print( "Multiplicación A X B: " )
print( M_cpu )
#"""

tiempo_total = datetime.now() - tiempo_total

print( "Cantidad de elementos: ", cantidad_N )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )


---
# 4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa los vectores A, B y M.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria M a CPU memoria M.
CPU      |  print()               | Informo los resultados.



---
# 5 Conclusiones

A medida que aumenta la cantidad de elementos (N) del vector aumenta el tiempo de procesamiento. A pesar de que en GPU tenemos más funciones y pasos para ejecutar (ej. reservar la memoria en area de GPU, copia de datos, definir y compilar Kernel, ejecución de Kernel) podemos notar que al realizar la multiplicación de dos vectores de N elementos hacerlo con GPU reduce mucho el tiempo total comparado con el tiempo total de la misma operación solo con CPU. Esto ultimo se debe a que en GPU la ejecución se realiza mediante hilos concurrentes.

---
# 6 Bibliografia

Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)



